In [1]:
from functions import *
trigger_words = ['data', 'machine learning', 'ai', 'ml ', 'statist', 'artificial intelligence', 'python']
#trigger_words = ['environment', 'climate', 'sustainability', 'green', 'renewable energy', 'conservation', 'legal', 'law', 'policy', 'regulation', 'compliance', 'environmental law', 'environmental policy', 'environmental regulation']
keyword = 'Data science'
skills = ['python', 'sql', 'machine learning', 'data analysis', 'statistics', 'matplotlib', 'pandas',
    'numpy', 'scikit-learn', 'tensorflow', 'pytorch', 'keras', 'data visualization', 'data wrangling', 
    'big data', 'cloud computing', 'pyspark', 'hadoop', 'spark', 'tableau', 'power bi',
    'data mining', 'data engineering', 'data modeling', 'data governance', 'data quality',
    'data architecture', 'data strategy', 'data storytelling', 'data ethics', 'data privacy',
    'data security', 'data compliance', 'data management', 'data integration', 'data transformation',
    'data pipeline', 'data lake', 'data warehouse', 'data mart', 'data catalog', 'data lineage',
    'data profiling', 'data cleansing', 'data enrichment', 'data visualization tools',
    'business intelligence tools', 'machine learning algorithms', 'deep learning algorithms',
    'natural language processing (NLP)', 'computer vision', 'reinforcement learning']

job_ids = get_job_ids(trigger_words, keyword, search_count=350, headers=None, internship=False)
data = fetch_job_details(job_ids, headers=None)
df = job_info_extractor(data, skills = skills)
# Save the data to CSV
save_jobs(df, 'data scientist', True)

Data fetched successfully!
!!! Job ID Data Scientist contains trigger words.
--- Job ID Data Science Intern(Analytics) is an intern position.
--- Job ID Internship - Data Science (HIVE) is an intern position.
--- Job ID Machine Learning Engineer - Internship is an intern position.
--- Job ID Data Science Intern (ML/AI) 104929 is an intern position.
--- Job ID Data Science Intern (ML/AI) is an intern position.
!!! Job ID Machine Learning Engineer contains trigger words.
--- Job ID R&D New Product Development Intern is an intern position.
!!! Job ID Machine Learning Engineer contains trigger words.
--- Job ID R&D Modelling-Statistics Intern is an intern position.

Data fetched successfully!
--- Job ID Ice Cream: Customer Development Internship – Data Analysis and Report is an intern position.
Job ID Associate,  Revenue Analytics - EMEA does not contain trigger words.
--- Job ID Internship - Business Intelligence is an intern position.
--- Job ID Data Science Intern is an intern position.

ValueError: Length of values (74) does not match length of index (71)

In [ ]:
save_jobs(data, keyword, date=True)

In [ ]:
# After reviewing the jobs, you can use the following function to blacklist job IDs that are not relevant or duplicates.
rows = None
cleanup = True
csv_file = 'data_scientist_jobs_24-04-2025.csv'
blacklist_job_ids(csv_file, rows=rows, cleanup=cleanup)

Found 0 outdated job ids in the blacklist.
No outdated job ids in the blacklist.


Job postings saved to Data_science_jobs_24-04-2025.csv
